# Variable-area flows: introduction



In [2]:
# Necessary modules to solve problems
import numpy as np
from scipy.optimize import root_scalar

%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
# these lines are only for helping improve the display
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'png')
plt.rcParams['figure.dpi']= 150
plt.rcParams['savefig.dpi'] = 150

## Equations for perfect gases

$$
\frac{A_2}{A_1} = \frac{M_1}{M_2} \left[ \frac{1 + \frac{\gamma-1}{2} M_2^2}{1 + \frac{\gamma-1}{2} M_1^2} \right]^{\frac{\gamma+1}{2(\gamma-1)}} e^{\Delta s/R}
$$

### Example: isentropic flow

**Problem:** Air flows isentropically through a duct ($\gamma = 1.4$) where the area is changing from point 1 to 2, 
with no heat transfer or shaft work. The area ratio is $\frac{A_2}{A_1} = 2.5$, the flow starts at $M_1 = 0.5$ and 4 bar.
Find the Mach number and pressure at the second point in the duct.

We can solve this using the classical approach (pre-calculated isentropic tables) or a numerical approach;
both follow the same general approach:
1. Find $M_2$ associated with the area ratio $A_2 / A_2^*$, then
2. Use that to find the stagnation pressure ratio $p_2 / p_{t2}$.

$$
\frac{A_2}{A_2^*} = \frac{A_2}{A_1} \frac{A_1}{A_1^*} \frac{A_1^*}{A_2^*} \;,
$$

where $\frac{A_2}{A_1} = 2.5$ is given, we can find $\frac{A_1}{A_1^*}$ using

$$
\frac{A}{A^*} = \frac{1}{M} \left( \frac{1 + \frac{\gamma - 1}{2} M^2}{\frac{\gamma+1}{2}} \right)^{\frac{\gamma+1}{2(\gamma-1)}} \;,
$$

(either by calculating or looking up in the $\gamma = 1.4$ table) 
and $\frac{A_1^*}{A_2^*} = 1$ because the flow is isentropic.

In [4]:
gamma = 1.4
mach_1 = 0.5

A2_A1 = 2.5
A1star_A2star = 1.0 # isentropic

A1_A1star = (1.0/mach_1) * ((1 + 0.5*(gamma-1)*mach_1**2) / ((gamma + 1)/2))**((gamma+1) / (2*(gamma-1)))
print(f'A1/A1^* = {A1_A1star:.4f}')

A1/A1^* = 1.3398


In [5]:
A2_A2star = A2_A1 * A1_A1star * A1star_A2star
print(f'A2/A2star = {A2_A2star:.4f}')

A2/A2star = 3.3496


We can then find $M2$, because $\frac{A_2}{A_2*} = f(M_2)$. 
Our options are to use the $\gamma = 1.4$ tables and interpolate, or solve the associated equation numerically.

**Using tables:** We can find in the tables that:
* at $M=0.17$, $A/A^* = 3.46351$
* at $M = 0.18$, $A/A^* = 3.27793$

and interpolate to find the precise $M_2$:

In [6]:
machs = np.array([0.17, 0.18])
areas = np.array([3.46351, 3.27793])
mach_2 = (machs[0] * (areas[1] - A2_A2star) + machs[1] * (A2_A2star - areas[0])) / (areas[1] - areas[0])
print(f'M2 = {mach_2:.4f}')

M2 = 0.1761


This is probably sufficient, but we could get a more-accurate result by interpolating using more points and using the `numpy.interp()` function:

In [7]:
machs = np.array([0.15, 0.16, 0.17, 0.18, 0.19])
areas = np.array([3.91034, 3.67274, 3.46351, 3.27793, 3.11226])

mach_2 = np.interp(A2_A2star, areas[::-1], machs[::-1])
print(f'M2 = {mach_2:.4f}')

M2 = 0.1761


Note that we have to reverse the order of the values, since `interp` expects the x-values to be increasing.
Also, we could easily generate these values ourselves for a different value of $\gamma$, but it is likely
easier to just solve the equation directly in that case

**Using the equation:** Alternately, we can solve the equation directly using `scipy.optimize.root_scalar`:

In [8]:
def area_function(mach, gamma, area_ratio):
    '''Function for area ratio, solving for M2'''
    return area_ratio - ((1.0/mach) * ((1 + 0.5*(gamma-1)*mach**2) / ((gamma + 1)/2))**((gamma+1) / (2*(gamma-1))))

sol = root_scalar(area_function, args=(gamma, A2_A2star), x0=0.1, x1=0.5)
print(f'M2 = {sol.root:.4f}')

M2 = 0.1760
